https://docs.celeryproject.org/en/latest/getting-started/brokers/index.html#broker-overview  

|Name|Status|Monitoring|Remote Control|
|---|---|---|---|
|RabbitMQ|Stable|Yes|Yes|
|Redis|Stable|Yes|Yes|
|Amazon SQS|Stable|No|No|
|Zookeeper|Experimental|No|No|

https://docs.celeryproject.org/en/latest/userguide/tasks.html#task-result-backends  
built-in: SQLAlchemy/Django ORM, Memcached, RabbitMQ/QPid (rpc), and Redis

https://docs.celeryproject.org/en/latest/getting-started/first-steps-with-celery.html#application

    celery -A tasks worker --loglevel=info  # restart if changes

https://flower.readthedocs.io/en/latest/install.html#usage

    flower -A tasks --port=5555 # http

In [97]:
!cat tasks.py

from celery import Celery

app = Celery('tasks', backend='amqp', broker='amqp://guest@localhost//') # backend: optional

import time

@app.task
def add(x, y, sleep_time=1):
    print(f"sleep {sleep_time}s") # no output, use logging instead
    time.sleep(sleep_time)
    return x + y



In [89]:
%load_ext autoreload
%autoreload 2
    
from tasks import *
import sys, datetime

result = add.delay(4, 4, sleep_time=3)
print(result.ready())
try:
    result.get(timeout=1)
except:
    print("Exception: ", sys.exc_info()[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
False
Exception:  <class 'celery.exceptions.TimeoutError'>


In [90]:
result = add.delay(4, 4, sleep_time=5)
print(result.ready())
try:
    print(datetime.datetime.now())
    result.get()
    print(datetime.datetime.now())
    print(result.ready())
except:
    print("Exception: ", sys.exc_info()[0])

False
2019-05-02 13:06:12.536567
2019-05-02 13:06:17.547183
True


In [77]:
result.get()

8

In [71]:
app.backend

In [68]:
app.broker_connection()

<Connection: amqp://guest:**@localhost:5672// at 0x7fbd94a72278>

In [91]:
app.tasks

{'celery.accumulate': <@task: celery.accumulate of tasks at 0x7fbd94a64940>,
 'celery.backend_cleanup': <@task: celery.backend_cleanup of tasks at 0x7fbd94a64940>,
 'celery.chain': <@task: celery.chain of tasks at 0x7fbd94a64940>,
 'celery.chord': <@task: celery.chord of tasks at 0x7fbd94a64940>,
 'celery.chord_unlock': <@task: celery.chord_unlock of tasks at 0x7fbd94a64940>,
 'celery.chunks': <@task: celery.chunks of tasks at 0x7fbd94a64940>,
 'celery.group': <@task: celery.group of tasks at 0x7fbd94a64940>,
 'celery.map': <@task: celery.map of tasks at 0x7fbd94a64940>,
 'celery.starmap': <@task: celery.starmap of tasks at 0x7fbd94a64940>,
 'tasks.add': <@task: tasks.add of tasks at 0x7fbd94a64940>}

In [92]:
app.tasks['tasks.add']

<@task: tasks.add of tasks at 0x7fbd94a64940>